In [1]:
from sqlalchemy import Column
from sqlalchemy import ForeignKey
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship

Base = declarative_base()

class User(Base):
    __tablename__ = "user_account"
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship(
        "Address", back_populates="user", cascade="all, delete-orphan"
    )
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"), nullable=False)
    user = relationship("User", back_populates="addresses")
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [2]:
from sqlalchemy import create_engine
engine = create_engine("sqlite://", echo=True, future=True)

In [3]:
Base.metadata.create_all(engine)

2022-07-20 10:39:07,992 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-20 10:39:07,993 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-07-20 10:39:07,994 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-20 10:39:07,997 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-07-20 10:39:07,998 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-20 10:39:08,000 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-07-20 10:39:08,001 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-20 10:39:08,004 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-07-20 10:39:08,005 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-20 10:39:08,007 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-07-20 10:39:08,009 INFO sqlalchemy.engine.Engine [no key 0.00135s] ()
2022-07-20 10:39:08,012 INFO sqlalchemy.engine.Engine 
C

In [4]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )
    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )
    patrick = User(name="patrick", fullname="Patrick Star")
    session.add_all([spongebob, sandy, patrick])
    session.commit()

2022-07-20 10:39:09,313 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-20 10:39:09,317 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-07-20 10:39:09,319 INFO sqlalchemy.engine.Engine [generated in 0.00195s] ('spongebob', 'Spongebob Squarepants')
2022-07-20 10:39:09,324 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-07-20 10:39:09,325 INFO sqlalchemy.engine.Engine [cached since 0.007787s ago] ('sandy', 'Sandy Cheeks')
2022-07-20 10:39:09,326 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-07-20 10:39:09,328 INFO sqlalchemy.engine.Engine [cached since 0.01051s ago] ('patrick', 'Patrick Star')
2022-07-20 10:39:09,332 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2022-07-20 10:39:09,334 INFO sqlalchemy.engine.Engine [generated in 0.00177s] ('spongebob@sqlalchemy.org', 1)
2022-07-20 10:39:09,336 INFO sqlalchemy.

In [5]:
from sqlalchemy import select

session = Session(engine)

stmt = select(User).where(User.name.in_(["spongebob", "sandy"]))

for user in session.scalars(stmt):
    print(user)

2022-07-20 10:39:10,622 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-20 10:39:10,626 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name IN (?, ?)
2022-07-20 10:39:10,628 INFO sqlalchemy.engine.Engine [generated in 0.00236s] ('spongebob', 'sandy')
User(id=1, name='spongebob', fullname='Spongebob Squarepants')
User(id=2, name='sandy', fullname='Sandy Cheeks')


In [6]:
stmt = (
 select(Address)
 .join(Address.user)
 .where(User.name == "sandy")
 .where(Address.email_address == "sandy@sqlalchemy.org")
)
sandy_address = session.scalars(stmt).one()


sandy_address

2022-07-20 10:39:17,719 INFO sqlalchemy.engine.Engine SELECT address.id, address.email_address, address.user_id 
FROM address JOIN user_account ON user_account.id = address.user_id 
WHERE user_account.name = ? AND address.email_address = ?
2022-07-20 10:39:17,721 INFO sqlalchemy.engine.Engine [generated in 0.00232s] ('sandy', 'sandy@sqlalchemy.org')


Address(id=2, email_address='sandy@sqlalchemy.org')

In [7]:
stmt = select(User).where(User.name == "patrick")
patrick = session.scalars(stmt).one()


patrick.addresses.append(
    Address(email_address="patrickstar@sqlalchemy.org")
)


sandy_address.email_address = "sandy_cheeks@sqlalchemy.org"

session.commit()


2022-07-20 10:39:28,051 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-07-20 10:39:28,053 INFO sqlalchemy.engine.Engine [generated in 0.00203s] ('patrick',)
2022-07-20 10:39:28,059 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2022-07-20 10:39:28,060 INFO sqlalchemy.engine.Engine [generated in 0.00104s] (3,)
2022-07-20 10:39:28,064 INFO sqlalchemy.engine.Engine UPDATE address SET email_address=? WHERE address.id = ?
2022-07-20 10:39:28,065 INFO sqlalchemy.engine.Engine [generated in 0.00110s] ('sandy_cheeks@sqlalchemy.org', 2)
2022-07-20 10:39:28,067 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2022-07-20 10:39:28,068 INFO sqlalchemy.engine.Engine [cached since 18.74s ago] ('patrickstar@sqlalchemy.org', 3

In [8]:
sandy = session.get(User, 2)


sandy.addresses.remove(sandy_address)


2022-07-20 10:39:38,246 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-20 10:39:38,250 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2022-07-20 10:39:38,253 INFO sqlalchemy.engine.Engine [generated in 0.00301s] (2,)
2022-07-20 10:39:38,257 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2022-07-20 10:39:38,258 INFO sqlalchemy.engine.Engine [cached since 10.2s ago] (2,)


In [9]:
session.flush()

2022-07-20 10:39:52,467 INFO sqlalchemy.engine.Engine DELETE FROM address WHERE address.id = ?
2022-07-20 10:39:52,470 INFO sqlalchemy.engine.Engine [generated in 0.00303s] (2,)


In [10]:
session.delete(patrick)

2022-07-20 10:40:05,858 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2022-07-20 10:40:05,861 INFO sqlalchemy.engine.Engine [cached since 27.61s ago] (3,)
2022-07-20 10:40:05,865 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2022-07-20 10:40:05,866 INFO sqlalchemy.engine.Engine [cached since 37.81s ago] (3,)


In [11]:
session.commit()


2022-07-20 10:40:12,860 INFO sqlalchemy.engine.Engine DELETE FROM address WHERE address.id = ?
2022-07-20 10:40:12,865 INFO sqlalchemy.engine.Engine [cached since 20.4s ago] (4,)
2022-07-20 10:40:12,868 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.id = ?
2022-07-20 10:40:12,869 INFO sqlalchemy.engine.Engine [generated in 0.00155s] (3,)
2022-07-20 10:40:12,873 INFO sqlalchemy.engine.Engine COMMIT
